# Описание проекта

Вы работаете в добывающей нефтяной компании. Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. 
Задача - построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

### Описание данных

Данные геологоразведки трёх регионов находятся в файлах:

* 		geo_data_0.csv
* 		geo_data_1.csv
* 		geo_data_2.csv
* 		`id` — уникальный идентификатор скважины;
* 		`f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
* 		`product` — объём запасов в скважине (тыс. баррелей).

### Условия задачи:

* 		Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* 		При разведке региона исследуют 500 точек, из которых выбирают 200 лучших для расчёта прибыли.
* 		Бюджет на разработку скважин в регионе — 10 млрд рублей.
* 		Один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* 		После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.


# 1. Загрузка и подготовка данных

In [1]:
# Импортируем необходимые библиотеки

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from scipy import stats as st
import warnings
warnings.filterwarnings('ignore')

# Читаем исходные файлы
data_reg_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_reg_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_reg_2 = pd.read_csv('/datasets/geo_data_2.csv')

# Смотрим общую информацию по каждому файлу + пропуски + дубликаты

def info_func(frame_name):
    print('INFO')
    display(frame_name.info())
    print('HEAD 10')
    display(frame_name.head(10))
    print('DESCRIBE')
    display(frame_name.describe())
    print('SPACES')
    display(frame_name.isnull().sum())
    print('DUPLICATES')
    display(frame_name.duplicated().sum())

### Датасет  `data_reg_0`

In [2]:
info_func(data_reg_0)

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

HEAD 10


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


DESCRIBE


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


SPACES


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

DUPLICATES


0

### Датасет  `data_reg_1`

In [3]:
info_func(data_reg_1)

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

HEAD 10


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


DESCRIBE


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


SPACES


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

DUPLICATES


0

### Датасет  `data_reg_2`

In [4]:
info_func(data_reg_2)

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

HEAD 10


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


DESCRIBE


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


SPACES


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

DUPLICATES


0

Столбец `id`, во всех трех исходных датасетах, не понадобится нам в дальнейшей работе. Избавимся от него.

In [5]:
data_reg_0 = data_reg_0.drop(['id'], axis = 1)
data_reg_1 = data_reg_1.drop(['id'], axis = 1)
data_reg_2 = data_reg_2.drop(['id'], axis = 1)

# 1. Загрузка и подготовка данных. Выводы.

- каждый из исходных датафреймов представлен 10 000 записями и 5 признаками
- в связи с ненужностью в дальнейшей работе, признак `id` удален из каждого датафрейма
- тип данных признаков(float64) подходит для дальнейших, необходимых в ходе проекта, вычислений
- пробелы и дубликаты не обнаружены

Данные готовы для дальнейшейй работы.

# 2. Обучение и проверка модели

### Датасет  `data_reg_0`
### Датасет  `data_reg_1`
### Датасет  `data_reg_2`

Создадим функцию, которая для каждого региона:
 - разделит на обучающую и валидационную выборки, в соотношении 75:25 соответственно
 - создаст переменные для признаков и целевого признака
 - создаст и обучит модель
 - оценит качество модели
 - рассчитает средний запас сырья

In [21]:
data_all = [data_reg_0, data_reg_1, data_reg_2]
all_target = []
all_predictions = []
for region in range(len(data_all)):
    data = data_all[region]
    features = data.drop('product', axis=1)
    target = data['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state = 12345)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    
    all_target.append(target_valid.reset_index(drop=True))
    all_predictions.append(pd.Series(predictions))
    
    mean_product = target.mean()
    model_rmse = mean_squared_error(target_valid, predictions)**0.5
    
    print("-- Регион", region, "--")
    print("Средний запас сырья = {:.2f}".format(mean_product),  "тыс.баррелей")
    print("RMSE модели(valid): {:.2f}".format(model_rmse))
    print()

-- Регион 0 --
Средний запас сырья = 92.50 тыс.баррелей
RMSE модели(valid): 37.58

-- Регион 1 --
Средний запас сырья = 68.83 тыс.баррелей
RMSE модели(valid): 0.89

-- Регион 2 --
Средний запас сырья = 95.00 тыс.баррелей
RMSE модели(valid): 40.03



# 2. Обучение и проверка модели. Выводы.

- исходные датасеты были поделены на обучающую и валидационную выборки в соотношении 75:25
- к выборкам был применен алгоритм линейной регрессии (`LinearRegression`)
- в результате обучения:
    - наилучший показатель качества модели получил датасет `data_reg_1`, с показателем RMSE = `0.89`
    - наихудший показатель у датасета `data_reg_2`, с показателем RMSE = `40.03`
    - наибольший средний запас скважины имеет регион №2 (`95.00` тыс баррелей)
    - наименьший средний запас скважины у регион №1 (`68.83` тыс баррелей)

# 3. Подготовка к расчёту прибыли

Сохраним ключевые значения для рассчета прибыли в константах

In [22]:
INTELL_POINTS = 500 # Количество точек для разведки в регионе
BUDGET = 10000000000 # Бюджет на разработку месторождений
COST_PER_POINT = 50000000 # Стоимость бурения одной скважины
PROFIT_PER_BRL = 450 # Доход с одного барреля

Оценим максимальное количество скважин которые можно открыть при текущем бюджете

In [23]:
open_points_count = BUDGET / COST_PER_POINT
print("Максимальное количество скважин для разработки = {:.0f}".format(open_points_count), "скважин")

Максимальное количество скважин для разработки = 200 скважин


Рассчитаем минимальный средний объём сырья в месторождениях региона, достаточный для его разработки, и сравним его со средними запасами в каждом регионе

In [24]:
field_min_avg = COST_PER_POINT / (PROFIT_PER_BRL * 1000)
print("Минимальный средний запас сырья = {:.2f}".format(field_min_avg), "тыс.баррелей")
print("Средний запас скважин в Регионе_0 = {:.2f}".format(data_reg_0['product'].mean()), "тыс.баррелей")
print("Средний запас скважин в Регионе_1 = {:.2f}".format(data_reg_1['product'].mean()), "тыс.баррелей")
print("Средний запас скважин в Регионе_2 = {:.2f}".format(data_reg_2['product'].mean()), "тыс.баррелей")

Минимальный средний запас сырья = 111.11 тыс.баррелей
Средний запас скважин в Регионе_0 = 92.50 тыс.баррелей
Средний запас скважин в Регионе_1 = 68.83 тыс.баррелей
Средний запас скважин в Регионе_2 = 95.00 тыс.баррелей


Создадим функцию для рассчета прибыли по выбранным скважинам и предсказаниям модели

In [25]:
def revenue(target, predicted, count):
    predicted_sorted = predicted.sort_values(ascending = False)
    selected = target[predicted_sorted.index][:count]
    return ((PROFIT_PER_BRL * selected.sum() * 1000) - (COST_PER_POINT * count)) / 1000000000

# 3. Подготовка к расчёту прибыли. Вывод.

- минимально достаточным объёмом сырья для безубыточной разработки новой скважины, в среднем, является объем = `111.11` тыс.баррелей
- ни один из рассматриваемых регионов не обеспечивает минимально достаточного объёма сырья для безубыточной разработки:
    - `регион_0` = `92.50` тыс.баррелей
    - `регион_1` = `68.83` тыс.баррелей
    - `регион_2` = `95.00` тыс.баррелей

# 4. Расчёт прибыли и рисков

### Датасет  `data_reg_0`
### Датасет  `data_reg_1`
### Датасет  `data_reg_2`

Создадим функцию для рассчета прибыли и рисков (доля отрицательных значений)

In [26]:
def func_rev_and_risk(predictions_valid_x, target_valid_x):
    # Проведем бутстреп с 1000 выборок
    values = []
    state = np.random.RandomState(12345)
    predictions_valid_x = pd.Series(np.array(predictions_valid_x))
    target_valid_x = target_valid_x.reset_index(drop = True) 
    for i in range(1000):
        target_subsample = target_valid_x.sample(500, replace = True, random_state = state)
        predict_subsample = predictions_valid_x[target_subsample.index]
        values.append(revenue(target_subsample, predict_subsample, 200))

    values = pd.Series(values)
    # Рассчитаем долю отрицательных значений прибыли
    lower = st.percentileofscore(values, 0)
    # Рассчитаем среднюю прибыль
    mean = values.mean()
    # Рассчитаем 95% доверительный интервал для истинного среднего
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
    # Рассчитаем 95% доверительный интервал для прогнозных значений
    confidence_interval_forecast = (values.quantile(0.025),values.quantile(0.975))
    
    print("Средняя прибыль 200 скважин региона: {:.3f}".format(mean), "млрд.руб")
    print("95%-ый доверительный интервал по региону (по истинному среднему):", confidence_interval)
    print("95%-ый доверительный интервал по региону (по прогнозным значениям):", confidence_interval_forecast)
    print('Риск убытков =', lower, '%')

Рассчитаем прибыль и риски для каждого из регионов

In [27]:
for i in range(3):
    print("--Регион", i,"--")
    print()
    func_rev_and_risk(all_predictions[i], all_target[i])
    print()

--Регион 0 --

Средняя прибыль 200 скважин региона: 0.426 млрд.руб
95%-ый доверительный интервал по региону (по истинному среднему): (0.4087322070686986, 0.4431448467524861)
95%-ый доверительный интервал по региону (по прогнозным значениям): (-0.10209009483793655, 0.9479763533583689)
Риск убытков = 6.0 %

--Регион 1 --

Средняя прибыль 200 скважин региона: 0.515 млрд.руб
95%-ый доверительный интервал по региону (по истинному среднему): (0.5016214761763218, 0.528824070710258)
95%-ый доверительный интервал по региону (по прогнозным значениям): (0.06887322537050176, 0.9315475912570494)
Риск убытков = 1.0 %

--Регион 2 --

Средняя прибыль 200 скважин региона: 0.435 млрд.руб
95%-ый доверительный интервал по региону (по истинному среднему): (0.4174535520413577, 0.4525631735241533)
95%-ый доверительный интервал по региону (по прогнозным значениям): (-0.128880547329789, 0.969706954180268)
Риск убытков = 6.4 %



# 5. Общий вывод.

### 5.1 Цели проекта

Для исследования предложены три региона с 3 признаками и целевым признаком в виде "объём запасов в месторождении (тыс. баррелей)".

Цели проектной работы:

- расчитать возможную прибыль каждого региона
- расчитать возможные риски каждого региона

### 5.2 Оценка регионов

Общая оценка регионов

- на разработку региона выделяется сумма `10 млрд рублей`. При затратах на одну скважину `50 млн руб` возможна разработка `200` скважин.
- минимальный средний обьем сырья в скважине, достаточный для её безубыточной разработки составляет `111.11` тыс. баррелей. Этому требованию не соответствуют все исследуемые регионы.
- только один регион, из анализируемых,а именно `регион №1`, удовлетворяет критерию оценки риска `< 2,5%`

#### 5.2.1 Регион_0
- cредняя прибыль 200 скважин региона: `0.426` млрд.руб
- 95%-ый доверительный интервал (по истинному среднему) = (`0.4087322070686986`, `0.4431448467524861`)
- 95%-ый доверительный интервал (по прогнозным значениям) = (`-0.10209009483793655`, `0.9479763533583689`)
- риск убытков = `6.0 %`

#### 5.2.2 Регион_1
- средняя прибыль 200 скважин региона: `0.515` млрд.руб 
- 95%-ый доверительный интервал (по истинному среднему) = (`0.5016214761763218`, `0.528824070710258`)
- 95%-ый доверительный интервал (по прогнозным значениям) = (`0.06887322537050176`, `0.9315475912570494`)
- риск убытков = `1.0 %`

#### 5.2.3 Регион_2
- cредняя прибыль 200 скважин региона: `0.435` млрд.руб
- 95%-ый доверительный интервал (по истинному среднему) = (`0.4174535520413577`, `0.4525631735241533`)
- 95%-ый доверительный интервал (по прогнозным значениям) = (`-0.128880547329789`, `0.969706954180268`)
- риск убытков = `6.4 %`

### 5.3 Рекомендации

Приоритетным для разработки предлагается `регион №1`.

#### Обоснование:

- регион имеет максимальную среднюю прибыль с 200 скважин (`0.515` млрд.руб)
- у региона минимальная (относительно других регионов) отрицательная прибыль
- нижняя граница 95% доверительного интервала выше верхних границ аналогичных интервалов других регионов
- предиктивные данные по этому региону имеют максимальное качество модели по показателю RMSE (`0,89`)

При этом необходимо учитывать, что `регион №1` имеет минимальный среди всех остальных средний запас скважины:
- средний запас скважин в `регионе №0` = `92.50` тыс.баррелей
- средний запас скважин в `регионе №1` = `68.83` тыс.баррелей
- средний запас скважин в `регионе №2` = `95.00` тыс.баррелей


===================================================================================================================


#### Спасибо за внимание.